## Configuration
_Initial steps to get the notebook ready to play nice with our repository. Do not delete this section._

Code formatting with [black](https://pypi.org/project/nb-black/).

In [2]:
import os
import pytz
import glob
import pathlib

this_dir = pathlib.Path(os.path.abspath(""))
data_dir = this_dir / "data"

In [3]:
import requests
import pandas as pd
import json
from datetime import datetime

## Download

Retrieve the page

In [4]:
url = "https://services2.arcgis.com/22p1CUjMjjRWlw6O/arcgis/rest/services/Shasta_County_COVID_19_Rates_by_Region_WFL1/FeatureServer/1//query"

In [5]:
params = dict(f="json", where="0=0", outFields="*")

In [6]:
session = requests.Session()
r = session.get(url, headers={"User-Agent": "Mozilla/5.0"}, params=params)

In [7]:
data = r.json()

## Parse

In [8]:
dict_list = []

In [9]:
for item in data["features"]:
    d = dict(
        county="Shasta",
        area=item["attributes"]["Region"],
        confirmed_cases=item["attributes"]["Cases"],
    )
    dict_list.append(d)

In [10]:
df = pd.DataFrame(dict_list)

Get timestamp

In [11]:
date_url = "https://services2.arcgis.com/22p1CUjMjjRWlw6O/arcgis/rest/services/Shasta_County_COVID_19_Rates_by_Region_WFL1/FeatureServer/1"
params = dict(f="json")
date_r = session.get(date_url, headers={"User-Agent": "Mozilla/5.0"}, params=params)
date_data = date_r.json()

In [12]:
timestamp = date_data["editingInfo"]["lastEditDate"]

In [13]:
timestamp = datetime.fromtimestamp((timestamp / 1000))

In [14]:
latest_date = pd.to_datetime(timestamp).date()
latest_date

datetime.date(2021, 6, 29)

In [15]:
df["county_date"] = latest_date

In [16]:
df["county"] = "Shasta"

## Vet

In [17]:
try:
    assert not len(df) > 6
except AssertionError:
    raise AssertionError("Shasta County scraper has extra rows")

In [18]:
try:
    assert not len(df) < 6
except AssertionError:
    raise AssertionError("Shasta County scraper is missing rows")

## Export

Set date

In [19]:
tz = pytz.timezone("America/Los_Angeles")

In [20]:
today = datetime.now(tz).date()

In [21]:
slug = "shasta"

In [22]:
df.to_csv(data_dir / slug / f"{today}.csv", index=False)

## Combine

In [23]:
csv_list = [
    i
    for i in glob.glob(str(data_dir / slug / "*.csv"))
    if not str(i).endswith("timeseries.csv")
]

In [24]:
df_list = []
for csv in csv_list:
    if "manual" in csv:
        df = pd.read_csv(csv, parse_dates=["date"])
    else:
        file_date = csv.split("/")[-1].replace(".csv", "")
        df = pd.read_csv(csv, parse_dates=["county_date"])
        df["date"] = file_date
    df_list.append(df)

In [25]:
df = pd.concat(df_list).sort_values(["date", "area"])

In [26]:
df.to_csv(data_dir / slug / "timeseries.csv", index=False)